In [9]:
import numpy as np
from onix.units import ureg, Q_
from typing import Optional, Union
import csv

In [489]:
class Photodiode:
    """Photodiode shot noise calculation.

    Args:
        R: photocurrent responsivity at the target wavelength, A/W. This number does not account for amplification in an APD.
        M: APD photoelectron amplification factor. For other photodiodes, it should be 1.
        G: transimpedance gain from current to output voltage.
        maximum: Used to calculate the maximum input optical power depending on the input units:
                            - units of [electric potential], maximum is the maximum output voltage;
                            - units of [current], maximum is the maximum output current;
                            - units of [power], maximum is the maximum input optical power.
        NEP: based on Thorlabs definition (https://www.thorlabs.com/images/TabImages/Noise_Equivalent_Power_White_Paper.pdf)
             the input signal power that results in an SNR of 1 at 1 Hz output bandwidth. Thorlabs uses "optical NEP" but assumes
             it is equal to the "electrical NEP". Used to determine the electrical noise.
    """
    def __init__(self, R: Q_, M: float = 1, G: Q_ = 50 * ureg.V / ureg.A, maximum: Optional[Q_] = None):
    # def __init__(self, R: Q_, M: float, G: Q_, max_V_out: Optional[Q_] = None):
    # def __init__(self, R: Q_, M: float, G: Q_, max_V_out: Q_, NEP: Q_):
        self._P = None
        self._R = R
        self._M = M
        self._G = G

        self.max_P = self._max_P(maximum)

    def _max_P(self, maximum) -> Union[None, Q_]:
        """Maximum input optical power."""
        if maximum.check("[electric_potential]"):
            return (maximum / (self._G * self._M * self._R)).to("W")
        elif maximum.check("[current]"):
            return (maximum / (self._M * self._R)).to("W")
        elif maximum.check("[power]"):
            return maximum.to("W")
        else:
            return None
    
    @property
    def P(self):
        """Input power."""
        return self._P

    @P.setter
    def P(self, P: Q_):
        """Input power."""
        if self.max_P is not None and P > self.max_P:
            print("Set power is higher than the maximum input power.")
        self._P = P

    @property
    def I_p(self) -> Q_:
        """Photocurrent before amplification."""
        return (self._P * self._R).to("A")

    @property
    def I_amp(self) -> Q_:
        """Photodiode current after amplification."""
        return self.I_p * self._M

    @property
    def V_out(self) -> Q_:
        """Photodiode output voltage."""
        return self.I_amp * self._G

    @property
    def N_I_p(self) -> Q_:
        """Noise spectral density of the photocurrent."""
        return (np.sqrt(2 * (1.6e-19 * ureg.A * ureg.s) * self.I_p)).to("A/Hz^(1/2)")

    @property
    def N_I_amp(self) -> Q_:
        """Noise spectral density of the amplified photodiode current."""
        return (self._M * self.N_I_p).to("A/Hz^(1/2)")

    @property
    def N_V_out(self) -> Q_:
        """Noise spectral density of the photodiode output voltage."""
        return (self._G * self.N_I_amp).to("V/Hz^(1/2)")
    
    def electrical_noise(self, NEP_min: Q_, R_max: Q_, BW: Q_) -> Q_:
        """Electrical noise from NEP based on Thorlabs definition."""
        # NEP = NEP_min * R_max / self._R
        # return (NEP * np.sqrt(BW) * self._R * self._M *  self._G).to("V")
        return (NEP_min * R_max * self._M * np.sqrt(BW) * self._G).to("V")
        
    def SNR(self, BW: Q_) -> Q_:
        """SNR for a certain bandwidth."""
        return (self._G * self.I_amp / self.N_V_out / np.sqrt(BW)).to("").magnitude

In [494]:
test = Photodiode(8 * ureg.A / ureg.W / 10, 10, 50 * ureg.kV / ureg.A, 100 * ureg.mA)

In [497]:
NEP_max = 0.46 * ureg.pW / np.sqrt(1 * ureg.Hz)
R_max = 10 * ureg.A / ureg.W / 10
BW = 50 * ureg.MHz
V = test.electrical_noise(NEP_max, R_max, BW)
print(V)
P = V**2 / (50 * ureg.ohm)
P.to("dBm")

0.00162634559672906 volt


-42.76544327964813 <Unit('decibelmilliwatt')>

In [498]:
test.P = 1 * ureg.uW

In [475]:
test.SNR(1 * ureg.Hz)

866025.4037844386